In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
#import tensorflow as tf

In [2]:
train_orig = np.genfromtxt('train.csv', delimiter=',',skip_header=1)
test_orig = np.genfromtxt('test.csv', delimiter=',',skip_header=1)

In [3]:
train_y_orig = train_orig.T[0].reshape((1,train_orig.shape[0]))
train_y_orig = np.eye(10)[np.squeeze(train_y_orig).astype(int)].T
print "Train Y : " ,train_y_orig.shape

train_x_orig = train_orig.T[1:]
print "Train X : " ,train_x_orig.shape

test_x_orig = test_orig.T
print "Test X : ", test_x_orig.shape


Train Y :  (10, 42000)
Train X :  (784, 42000)
Test X :  (784, 28000)


In [4]:
def sigmoid(Z):
    return 1/(1+np.exp(-Z))

def sigmoid_prime(Z):
    return sigmoid(Z)*(1-sigmoid(Z))

def relu(Z):
    return np.maximum(0,Z)

def relu_prime(Z):
    if Z>0:
        return 1
    return 0

def softmax(Z):
    n = np.exp(Z)
    return n/np.sum(n, axis=1, keepdims=True)

In [5]:
def random_mini_batches(X,Y,mini_batch_size=64):
    m = X.shape[1]
    mini_batches = []
    shuffled_X = X
    shuffled_Y = Y
    #permutation = list(np.random.permutation(m))
    #shuffled_X = X[:, permutation]
    #shuffled_Y = Y[:, permutation].reshape((10,m))
    
    num_complete_minibatches = int(math.floor(m/mini_batch_size))
    
    for k in range(0,num_complete_minibatches):
        mini_batch_X = shuffled_X[:,k * mini_batch_size:(k + 1) * mini_batch_size]
        mini_batch_Y = shuffled_Y[:,k * mini_batch_size:(k + 1) * mini_batch_size]
        
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    if m % mini_batch_size != 0:
        end = m - mini_batch_size * math.floor(m / mini_batch_size)
        mini_batch_X = shuffled_X[:,num_complete_minibatches * mini_batch_size:]
        mini_batch_Y = shuffled_Y[:,num_complete_minibatches * mini_batch_size:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches



def initialize_parameters(nx,nh,ny):
    W1 = np.random.randn(nh,nx) * np.sqrt(2/nx)
    b1 = np.zeros((nh,1))
    W2 = np.random.randn(ny,nh) * np.sqrt(2/nh)
    b2 = np.zeros((ny,1))
    parameters = {"W1" : W1, "W2" : W2, "b1" : b1, "b2" : b2}
    return parameters


def forward_pass(parameters, X):
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    Z1 = np.dot(W1,X)+b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(W2,A1)+b2
    A2 = softmax(Z2)
    indices = np.argmax(A2, axis=0).reshape(-1)
    #print indices
    nb_classes = 10
    a2 = np.eye(nb_classes)[indices]
    A2 = a2.T
    cache = {"Z1" : Z1, "Z2":Z2, "A1":A1, "A2":A2}
    return A2, cache

def compute_cost(A2, Y):
    m = Y.shape[1]
    logprobs = np.multiply(np.log(A2), Y)
    print np.sum(logprobs)
    cost = - np.sum(logprobs) / m
    cost = np.squeeze(cost)
    print cost
    
    
def backprop(parameters, cache, X, Y):
    m = X.shape[1]
    W1 = parameters['W1']
    W2 = parameters['W2']
    A1 = cache['A1']
    A2 = cache['A2']
    dZ2= A2 - Y
    dW2 = (1 / m) * np.dot(dZ2, A1.T)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.multiply(np.multiply(np.dot(W2.T, dZ2), (1 - A1)), A1)
    dW1 = (1 / m) * np.dot(dZ1, X.T)
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

def update_parameters(parameters, grads, learning_rate):
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    
    W1 = W1 - learning_rate*dW1
    b1 = b1 - learning_rate*db1
    W2 = W2 - learning_rate*dW2
    b2 = b2 - learning_rate*db2
    
    parameters = {"W1" : W1, "W2" : W2, "b1" : b1, "b2" : b2}
    return parameters

In [6]:
def model(X, Y, learning_rate, mini_batch_size, num_epochs):
    nx = 784
    nh = 30
    ny = 10
    parameters = initialize_parameters(nx,nh,ny)
    
    for i in range(num_epochs):
        minibatches = random_mini_batches(X, Y, mini_batch_size)
                
        for minibatch in minibatches:
            (minibatch_X, minibatch_Y) = minibatch
            #print minibatch_X.shape, minibatch_Y.shape
            a2, cache = forward_pass(parameters, minibatch_X)
            cost = compute_cost(a2, minibatch_Y)
            grads = backprop(parameters, cache, minibatch_X, minibatch_Y)
            update_parameters(parameters, grads, learning_rate)
        
        if i%10==0:
            print("Cost after epoch %i: %f" % (i, cost))
    
    plt.plot(costs)
    plt.ylabel('cost')
    plt.xlabel('epochs (per 100)')
    plt.title("Learning rate = " + str(learning_rate))
    plt.show()

    return parameters

In [7]:
X = train_x_orig
Y = train_y_orig
learning_rate=3.0
mini_batch_size=32
num_epochs=1000
parameters = model(X,Y,learning_rate, mini_batch_size, num_epochs)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in multiply


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


TypeError: float argument required, not NoneType